In [10]:
%load_ext autoreload
%autoreload 2
import glob2
import pandas as pd
import numpy as np
import text_summarization
import extract_text
import sentiment_analyzer

#from tqdm import tqdm
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
def find_sentiment(path):
    try:
        text = extract_text.apply(path)
        important_sents = text_summarization.apply(text)
        #print(len(important_sents))
        importance = np.mean(sentiment_analyzer.apply(important_sents))
        return np.mean(importance)
    except:
        return -999

In [ ]:
from nltk import pos_tag, ne_chunk
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('maxent_ne_chunker')
nltk.download('words')
NER = ne_chunk(pos_tag(word_tokenize("test")))

In [ ]:
tqdm().pandas(desc="Progress:")
df_files['sentiment'] = df_files.progress_apply(lambda row: find_sentiment(row.file_path), axis=1)

In [ ]:
df_files

In [ ]:
def getSubmission(df):
    dfCleaned = df[df['sentiment']>0.01]
    top = list(dfCleaned.sort_values('sentiment',ascending=False).head(10).id)
    down = list(dfCleaned.sort_values('sentiment',ascending=True).head(10).id)
    return top,down

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
sentiment_analyzer.apply(['I am ugly', 'success'])